# ============ Podcast2Podcast ============

This notebook demonstrates a fully-automated pipeline to produce a podcast summary podcast from any podcast!

[Check out an example of this pipeline run against the NY Times' The Daily.](https://podcasts.apple.com/us/podcast/the-d-ai-ly-with-jeremybot/id1667416605)

## 📄 Instructions


1.   Fill out the configuration and run the "⚙️ Configuration" cell. This will provide an episode selection.
2.   Select your episode
3.   Run the remaining cells. This will take awhile, especially to get higher-quality results from TortoiseTTS.
4.   Listen to the results on the last cell of this notebook

In [ ]:
#@title ⚙️ Configuration

#@markdown **RSS URL:** If you don't know the RSS feed URL, Google `"<your-podcast> apple podcasts"` to find the iTunes page. Copy the iTunes page URL into [https://getrssfeed.com/](https://getrssfeed.com/). Then, right-click on "RSS feed" and select "Copy Link Address."
podcast_rss_feed_url = "https://lexfridman.com/feed/podcast/" #@param ["https://talkpython.fm/episodes/rss", "https://feeds.captivate.fm/gradient-dissent/", "http://rss.acast.com/mydadwroteaporno", "https://lexfridman.com/feed/podcast/", "https://feeds.simplecast.com/54nAGcIl"] {allow-input: true}

#@markdown **OpenAI API Token:** Go to [https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)
openai_token = "" #@param {type:"string"}

#@markdown **TortoiseTTS quality setting :**
tortoise_settings = "high_quality" #@param ["ultra_fast", "fast", "standard", "high_quality"]

#@markdown **Save output** The pipeline takes a long time and it is a good idea to save the output to Google Drive in case the colab interface gets disconnected.
save_to_google_drive = False #@param {type:"boolean"}
gdrive_export_fp = "/gdrive/MyDrive/podcast_summary.mp3" #@param {type:"string"}

if save_to_google_drive:
    from google.colab import drive
    drive.mount("/gdrive")

!pip install -qqq \
    git+https://github.com/jeremyadamsfisher/podcast2podcast \
    1> /dev/null 2> /dev/null

import os, ipywidgets

os.environ["OPENAI_API_KEY"] = openai_token

from podcast2podcast.rss import parse_rss

_, episodes = parse_rss(podcast_rss_feed_url)
episodes_titles, _ = zip(*episodes)


episode_title_selection = ipywidgets.Select(
    options=episodes_titles,
    value=episodes_titles[0],
    rows=15,
    description="Episode: ",
)

episode_title_selection

In [ ]:
#@title 👷‍♂️ Install Dependencies
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!(  git clone https://github.com/neonbjb/tortoise-tts.git \
 && cd tortoise-tts \
 && pip install --quiet -r requirements.txt \
 && python setup.py install ) \
  > /content/podcast2podcast.installs.log 2>&1
import os; os.chdir("/content/tortoise-tts/")

In [ ]:
#@title ⚡️ Run it!
%%time

from podcast2podcast import pipeline

audio = pipeline(
    podcast_rss_feed_url,
    episode_idx=episodes_titles.index(episode_title_selection.value),
    tts_method="tortoise",
)

if save_to_google_drive:
    audio.export(gdrive_export_fp, format="mp3")

audio